In [1]:
import os

from ourddpm import OurDDPM
from main import dict2namespace
import argparse
import yaml
from PIL import Image
import warnings

import torch
import pdb
import cv2
import glob
import pickle


%load_ext autoreload
%autoreload 2

warnings.filterwarnings(action='ignore')

device = 'cuda'

args_dic = {
    'config': 'celeba.yml',
    'bs_train': 4,
    'device': device
    }
args = dict2namespace(args_dic)

with open(os.path.join('configs', args.config), 'r') as f:
    config_dic = yaml.safe_load(f)
config = dict2namespace(config_dic)

runner = OurDDPM(args, config, device=device)

/home/summertony717/anaconda3/envs/ddpm-playground/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# torch.cuda.device_count()
runner.train_classifier()

Classifier created
Epoch 0


Epoch 0, Loss: 0.44:  12%|█▋             | 3359/29000 [07:54<1:00:19,  7.08it/s]


KeyboardInterrupt: 

In [8]:
from torch.nn.functional import sigmoid
import torch

a = torch.tensor([[1, 2],[2, 3]])
b = torch.tensor([[1,3],[2, 3]])
(a == b).sum(dim=1)

0 + torch.tensor([1, 2])

tensor([1, 2])